In [ ]:
""" Повезло мне с домашней работой, конечно...
Тут тебе и давольно большой датасет, огромный дисбаланс классов, да еще и с настройкой гиперпараметров надо поиграться....
Имеем датасет Credit Card Fraud Detection, который содержит переработанные с использованием PCA данные в числовом виде, где у нас есть 492 мошенничества из 284 807 транзакций (что в них было до переработки мы, конечно, не знаем)
Будем пробовать увеличивать количество мошенничеств с использованием SMOTE, попробуем уменьшить количество "хороших" транзакций с помощью Tomek Links, попробуем Boosting методы как наиболее устойчивые к дисбалансу и тд...
"""


In [2]:
import pandas as pd
df = pd.read_csv('creditcard.csv')

In [3]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [5]:
df['Class'].value_counts()

,count
Class,
0,284315
1,492


In [7]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
# обявим переменные
x = df.drop(columns='Class',axis=1)
y = df['Class']
# Разделение данных на обучающую и тестовую выборки
x_train, X_test, y_train, y_test = train_test_split(
x, y, test_size=0.2, random_state=0)

print("Raw class distribution:")
print(y_train.value_counts())
#теперь попробуем сгенерировать с помощью SMOTE данные во второй класс
smote = SMOTE(random_state=0)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

print("distribution after SMOTE oversampling:")
print(pd.Series(y_train_resampled).value_counts())
#теперь попробуем уменьшить данные первого класса с помощью TomekLinks
tomek = TomekLinks()
X_train_tomek, y_train_tomek = tomek.fit_resample(x_train, y_train)
print("distribution after TomekLinks undersampling:")
print(pd.Series(y_train_tomek).value_counts())

# TomekLinks очень долго работает и непонятно что в итоге с данными, т.к. количество почти не меняется, отказываемся от него, тем более в жизни его не применяют

Raw class distribution:
Class
0    227451
1       394
Name: count, dtype: int64
distribution after SMOTE oversampling:
Class
0    227451
1    227451
Name: count, dtype: int64
distribution after TomekLinks undersampling:
Class
0    227387
1       394
Name: count, dtype: int64


In [9]:
# попробуем GradientBoosting на данных без предобработки со стандартными гиперпараметрами, время выполнения 10 мин, видим переобучение по мажорному классу и ужас по минорному...
from sklearn.ensemble import GradientBoostingClassifier


# Создание модели Gradient Boosting Classifier
gb_classifier = GradientBoostingClassifier(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42)

# Обучение модели на обучающем наборе данных
gb_classifier.fit(x_train, y_train)

# Предсказание классов на тестовом наборе данных
y_pred = gb_classifier.predict(X_test)

# Вывод полного отчета
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.73      0.16      0.27        98

    accuracy                           1.00     56962
   macro avg       0.86      0.58      0.63     56962
weighted avg       1.00      1.00      1.00     56962



In [52]:
# попробуем GradientBoosting, но уже на данных после SMOTE
from sklearn.ensemble import GradientBoostingClassifier


# Создание модели Gradient Boosting Classifier
gb_classifier = GradientBoostingClassifier(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42)

# Обучение модели на обучающем наборе данных
gb_classifier.fit(x_train_resampled, y_train_resampled)

# Предсказание классов на тестовом наборе данных
y_pred = gb_classifier.predict(X_test)

# Вывод полного отчета
report_grad = classification_report(y_test, y_pred)
print(report_grad)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("AUC-ROC Score:", roc_auc_score(y_test, y_pred))
print("\n" + "="*50 + "\n")

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     56864
           1       0.20      0.84      0.32        98

    accuracy                           0.99     56962
   macro avg       0.60      0.92      0.66     56962
weighted avg       1.00      0.99      1.00     56962

Confusion Matrix:
[[56535   329]
 [   16    82]]
AUC-ROC Score: 0.9154744797468791




In [ ]:
# Ну все accuracy 0.99, премию мне)))) Ну а если серьёзно, то нагенерировали мы синтетических данных, поэтому и такой результат, да и CHATGPT говорит что при большом дисбалансе нужно ориентироваться на AUC-ROC, а не на accuracy,
# поэтому дальше будем пробовать играться с весами и дальше только андерсемплировать данные, а то 21 минута на обучение - такое себе ждать...

In [13]:
# поэтому, сейчас андерсемплируем данные и получаем всего 984 записи
majority_class = df[df["Class"] == 0]
minority_class = df[df["Class"] == 1]

#
majority_class_undersampled = majority_class.sample(len(minority_class), random_state=42)

#
df_undersampled = pd.concat([majority_class_undersampled, minority_class])

#
df_undersampled = df_undersampled.sample(frac=1, random_state=42).reset_index(drop=True)

#
print(df_undersampled)


# обявим переменные
x_und = df_undersampled.drop(columns='Class',axis=1)
y_und = df_undersampled['Class']
# Разделение данных на обучающую и тестовую выборки
x_train_und, X_test_und, y_train_und, y_test_und = train_test_split(
x_und, y_und, test_size=0.3, random_state=0)

         Time         V1        V2  ...       V28  Amount  Class
0     41233.0 -10.645800  5.918307  ... -0.152908    0.00      1
1      1308.0  -1.379848  0.536720  ... -0.346731   43.80      0
2     72327.0  -4.198735  0.194121  ... -0.131525  238.90      1
3     78685.0  -0.671155  1.183971  ...  0.055953    1.30      0
4     31244.0   1.087002  0.065459  ...  0.007649   18.56      0
..        ...        ...       ...  ...       ...     ...    ...
979   37976.0  -2.045484 -0.603428  ... -0.173156  350.00      0
980   60439.0  -0.377304  0.572787  ...  0.126249   12.95      0
981  128471.0   0.909124  1.337658  ...  0.319879   67.90      1
982  122517.0   1.554783 -1.059409  ... -0.008578  229.88      0
983   81324.0  -2.373043  2.563438  ... -0.148523   24.18      0

[984 rows x 31 columns]


In [44]:
# ну и теперь еще раз запустим градиент бустинг с автоподбором гиперпараметров
from sklearn.model_selection import GridSearchCV

# Определение параметров для подбора
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 500],
    'max_depth': [-1, 3, 10]
}
model = GradientBoostingClassifier()
# Подбор гиперпараметров с помощью GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(x_train_und, y_train_und)

# Лучшие параметры
best_params = grid_search.best_params_
print(f'Best parameters found: {best_params}')


# Создание модели Gradient Boosting Classifier
gb_classifier = GradientBoostingClassifier(**best_params)

# Обучение модели на обучающем наборе данных
gb_classifier.fit(x_train_und, y_train_und)

# Предсказание классов на тестовом наборе данных
y_pred_und = gb_classifier.predict(X_test_und)

# Вывод полного отчета
report_grad_CV = classification_report(y_test_und, y_pred_und)
print(report_grad_CV)
print("Confusion Matrix:")
print(confusion_matrix(y_test_und, y_pred_und))

print("AUC-ROC Score:", roc_auc_score(y_test_und, y_pred_und))
print("\n" + "="*50 + "\n")

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
45 fits failed out of a total of 135.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/

Best parameters found: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}
              precision    recall  f1-score   support

           0       0.90      0.93      0.92       141
           1       0.93      0.91      0.92       155

    accuracy                           0.92       296
   macro avg       0.92      0.92      0.92       296
weighted avg       0.92      0.92      0.92       296

Confusion Matrix:
[[131  10]
 [ 14 141]]
AUC-ROC Score: 0.9193777167696179




In [45]:
#Light Gradient Boosting
import lightgbm as lgb

# Создание и обучение классификатора LightGBM
clf = lgb.LGBMClassifier(num_leaves=31, learning_rate=0.05, n_estimators=100)
clf.fit(x_train_und, y_train_und)

# Прогнозирование и оценка точности
y_pred_und = clf.predict(X_test_und)

# Вывод полного отчета
report_lgb = classification_report(y_test_und, y_pred_und)
print(report_lgb)
print("Confusion Matrix:")
print(confusion_matrix(y_test_und, y_pred_und))

print("AUC-ROC Score:", roc_auc_score(y_test_und, y_pred_und))
print("\n" + "="*50 + "\n")

[LightGBM] [Info] Number of positive: 337, number of negative: 351
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000445 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6879
[LightGBM] [Info] Number of data points in the train set: 688, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.489826 -> initscore=-0.040703
[LightGBM] [Info] Start training from score -0.040703
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

In [53]:
#Light Gradient Boosting на полных данных с баллансировкой, получаем тоже очень красивый  accuracy
import lightgbm as lgb

# Создание и обучение классификатора LightGBM
clf = lgb.LGBMClassifier(num_leaves=31, learning_rate=0.05, n_estimators=100, class_weight='balanced')
clf.fit(x_train, y_train)

# Прогнозирование и оценка точности
y_pred_full = clf.predict(X_test)

# Вывод полного отчета
report_lgb_full = classification_report(y_test, y_pred)
print(report_lgb_full)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_full))

print("AUC-ROC Score:", roc_auc_score(y_test, y_pred_full))
print("\n" + "="*50 + "\n")

[LightGBM] [Info] Number of positive: 394, number of negative: 227451
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.072327 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7650
[LightGBM] [Info] Number of data points in the train set: 227845, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     56864
           1       0.20      0.84      0.32        98

    accuracy                           0.99     56962
   macro avg       0.60      0.92      0.66     56962
weighted avg       1.00      0.99      1.00     56962

Confusion Matrix:
[[56790    74]
 [   19    79]]
AUC-ROC Score: 0.9024105491943542




In [46]:
# AdaBoost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# Создание и обучение классификатора AdaBoost
base_estimator = DecisionTreeClassifier(max_depth=1)
ada_classifier = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)
ada_classifier.fit(x_train_und, y_train_und)

# Прогнозирование классов на тестовом наборе данных
y_pred_und = ada_classifier.predict(X_test_und)

# Вывод полного отчета
report_ada = classification_report(y_test_und, y_pred_und)
print(report_ada)
print("Confusion Matrix:")
print(confusion_matrix(y_test_und, y_pred_und))

print("AUC-ROC Score:", roc_auc_score(y_test_und, y_pred_und))
print("\n" + "="*50 + "\n")

              precision    recall  f1-score   support

           0       0.91      0.96      0.93       141
           1       0.96      0.91      0.93       155

    accuracy                           0.93       296
   macro avg       0.93      0.93      0.93       296
weighted avg       0.93      0.93      0.93       296

Confusion Matrix:
[[135   6]
 [ 14 141]]
AUC-ROC Score: 0.9335621139327386




In [47]:
#  QDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# Создание и обучение классификатора QDA
clf = QuadraticDiscriminantAnalysis()
clf.fit(x_train_und, y_train_und)

# Прогнозирование и оценка точности
y_pred_und = clf.predict(X_test_und)

# Вывод полного отчета
report_QDA = classification_report(y_test_und, y_pred_und)
print(report_QDA)
print("Confusion Matrix:")
print(confusion_matrix(y_test_und, y_pred_und))

print("AUC-ROC Score:", roc_auc_score(y_test_und, y_pred_und))
print("\n" + "="*50 + "\n")

              precision    recall  f1-score   support

           0       0.90      0.94      0.92       141
           1       0.94      0.91      0.92       155

    accuracy                           0.92       296
   macro avg       0.92      0.92      0.92       296
weighted avg       0.92      0.92      0.92       296

Confusion Matrix:
[[132   9]
 [ 14 141]]
AUC-ROC Score: 0.9229238160603981




/usr/local/lib/python3.11/dist-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 0 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(


In [48]:
# KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier

# Создание и обучение модели K Neighbors
knn = KNeighborsClassifier(n_neighbors=2)  # Задаем количество соседей (K=2), опытным путем получился самый лучший результат (можно еще K=7)
knn.fit(x_train_und, y_train_und)

# Предсказание на тестовом наборе
y_pred_und = knn.predict(X_test_und)

# Вывод полного отчета
report_KN = classification_report(y_test_und, y_pred_und)
print(report_KN)
print("Confusion Matrix:")
print(confusion_matrix(y_test_und, y_pred_und))

print("AUC-ROC Score:", roc_auc_score(y_test_und, y_pred_und))
print("\n" + "="*50 + "\n")

              precision    recall  f1-score   support

           0       0.57      0.79      0.66       141
           1       0.71      0.46      0.56       155

    accuracy                           0.62       296
   macro avg       0.64      0.63      0.61       296
weighted avg       0.64      0.62      0.61       296

Confusion Matrix:
[[112  29]
 [ 84  71]]
AUC-ROC Score: 0.626195378631892




In [49]:
# Экстремальный градиентный бустинг xgboost
import xgboost as xgb

# Создание и обучение модели XGBoost
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(x_train_und, y_train_und)
y_pred_und = model.predict(X_test_und)

# Вывод полного отчета
report_xgb = classification_report(y_test_und, y_pred_und)
print(report_xgb)
print("Confusion Matrix:")
print(confusion_matrix(y_test_und, y_pred_und))

print("AUC-ROC Score:", roc_auc_score(y_test_und, y_pred_und))
print("\n" + "="*50 + "\n")

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:50:46] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0       0.92      0.94      0.93       141
           1       0.95      0.92      0.93       155

    accuracy                           0.93       296
   macro avg       0.93      0.93      0.93       296
weighted avg       0.93      0.93      0.93       296

Confusion Matrix:
[[133   8]
 [ 12 143]]
AUC-ROC Score: 0.9329215282544041




In [50]:
# svm
from sklearn.svm import SVC
# Создание и обучение модели SVM
svm_model = SVC(kernel='poly', random_state=42) # альтернатива rbf, poly     rbf - 0,51
svm_model.fit(x_train_und, y_train_und)
y_pred_und = svm_model.predict(X_test_und)

# Вывод полного отчета
report_svm = classification_report(y_test_und, y_pred_und)
print(report_svm)
print("Confusion Matrix:")
print(confusion_matrix(y_test_und, y_pred_und))

print("AUC-ROC Score:", roc_auc_score(y_test_und, y_pred_und))
print("\n" + "="*50 + "\n")

              precision    recall  f1-score   support

           0       0.54      0.39      0.45       141
           1       0.56      0.70      0.62       155

    accuracy                           0.55       296
   macro avg       0.55      0.55      0.54       296
weighted avg       0.55      0.55      0.54       296

Confusion Matrix:
[[ 55  86]
 [ 46 109]]
AUC-ROC Score: 0.5466483642187143




In [51]:
#Выведем все отчеты
print("Отчет градиентный бустинг:")
print(report_grad_CV)
print("\n" + "="*50 + "\n")
print("Отчет Light Gradient Boosting:")
print(report_lgb)
print("\n" + "="*50 + "\n")
print("Отчет AdaBoost:")
print(report_ada)
print("\n" + "="*50 + "\n")
print("Отчет QDA:")
print(report_QDA)
print("\n" + "="*50 + "\n")
print("Отчет KNeighborsClassifier:")
print(report_KN)
print("\n" + "="*50 + "\n")
print("Отчет xgboost:")
print(report_xgb)
print("\n" + "="*50 + "\n")
print("Отчет svm:")
print(report_svm)
print("\n" + "="*50 + "\n")

Отчет градиентный бустинг:
              precision    recall  f1-score   support

           0       0.90      0.93      0.92       141
           1       0.93      0.91      0.92       155

    accuracy                           0.92       296
   macro avg       0.92      0.92      0.92       296
weighted avg       0.92      0.92      0.92       296



Отчет Light Gradient Boosting:
              precision    recall  f1-score   support

           0       0.91      0.94      0.92       141
           1       0.94      0.92      0.93       155

    accuracy                           0.93       296
   macro avg       0.93      0.93      0.93       296
weighted avg       0.93      0.93      0.93       296



Отчет AdaBoost:
              precision    recall  f1-score   support

           0       0.91      0.96      0.93       141
           1       0.96      0.91      0.93       155

    accuracy                           0.93       296
   macro avg       0.93      0.93      0.93       

In [ ]:
"""Как видим, все бустовые методы показали высокий accuracy на уровне 0,92-0,93. Но по показателю AUC-ROC в лидерах AdaBoost"""